In [1]:
import pandas as pd
import numpy as np

In [2]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [3]:
import requests
from requests_oauthlib import OAuth1

In [4]:
import cryptography
from cryptography.fernet import Fernet

In [51]:
from scipy.sparse import csr_matrix

In [55]:
from sklearn.externals import joblib

In [58]:
from sklearn.preprocessing import LabelEncoder

In [5]:
import datetime as dt

In [6]:
import random

In [7]:
import pickle

In [8]:
from Candidates import candidates_list

In [9]:
with open ('../config/config_key.key', 'rb') as ck:
    fernet_key = ck.read()

with open('../config/config_encrypt_1.key', 'rb') as c1:
    cke_e = c1.read()
with open('../config/config_encrypt_2.key', 'rb') as c2:
    cse_e = c2.read()
with open('../config/config_encrypt_3.key', 'rb') as c3:
    ate_e = c3.read()
with open('../config/config_encrypt_4.key', 'rb') as c4:
    atse_e = c4.read()

In [10]:
fernet = Fernet(fernet_key)

In [11]:
cke_d = fernet.decrypt(cke_e)
cse_d = fernet.decrypt(cse_e)
ate_d = fernet.decrypt(ate_e)
atse_d = fernet.decrypt(atse_e)

In [12]:
ck = cke_d.decode()
cs = cse_d.decode()
at = ate_d.decode()
ats = atse_d.decode()

In [13]:
# Create authorization object
auth = OAuth1(ck, cs, at, ats)

In [14]:
callback_url = "https://tweetocracy.herokuapp.com/"
payload = {
    'oauth_callback':callback_url
}

In [15]:
# Execute a POST/Auth request to twittier api to intitiate access
r = requests.post('https://api.twitter.com/oauth/request_token', auth = auth, data = payload)

In [16]:
print(f'Post Request Token URL:{r.url}')
print(f'Post Request Status:{r.status_code}')
print(f'Post Request Text: {r.text}')

Post Request Token URL:https://api.twitter.com/oauth/request_token
Post Request Status:200
Post Request Text: oauth_token=ztzmZAAAAAAA_QraAAABbNjtI-M&oauth_token_secret=b7vDVIEEhEGFaKF3xE9Kas06WsWC554u&oauth_callback_confirmed=true


In [17]:
# Collect response information
response_output = r.text
# Relevant paramters are received as a string, separated by an '&' character
response_parameters = response_output.split("&")

In [18]:
# Store relevant response paramters in variables
oauth_token = response_parameters[0][12:]
print(f'OAuth_token:{oauth_token}')
oauth_token_secret=response_parameters[1][19:]
print(f'Oauth Token Secret:{oauth_token_secret}')
oauth_callback_confirmed = bool(response_parameters[2][25:])
print(f'Callback Confirmed:{oauth_callback_confirmed}')

OAuth_token:ztzmZAAAAAAA_QraAAABbNjtI-M
Oauth Token Secret:b7vDVIEEhEGFaKF3xE9Kas06WsWC554u
Callback Confirmed:True


In [19]:
extended_payload = {
    'tweet_mode': 'extended'
}

In [20]:
with open('top_candidates.pkl', 'rb') as f:
   top_candidates = pickle.load(f)

In [21]:
top_candidates

['Donald J. Trump',
 'Bernie Sanders',
 'Kamala Harris',
 'Elizabeth Warren',
 'Joe Biden',
 'Tulsi Gabbard']

In [22]:
filtered_candidates = list(filter(lambda x: x["name"] in top_candidates, candidates_list))

In [23]:
filtered_candidates

[{'name': 'Joe Biden',
  'party': 'Democrat',
  'twitter_url': 'https://twitter.com/JoeBiden',
  'twitter_screen_name': 'JoeBiden',
  'twitter_user_id': '939091',
  'announcement_date': 'April 25, 2019',
  'status': 'running'},
 {'name': 'Tulsi Gabbard',
  'party': 'Democrat',
  'twitter_url': 'https://twitter.com/TulsiGabbard',
  'twitter_screen_name': 'TulsiGabbard',
  'twitter_user_id': '26637348',
  'announcement_date': 'January 12, 2019',
  'status': 'running'},
 {'name': 'Kamala Harris',
  'party': 'Democrat',
  'twitter_url': 'https://twitter.com/KamalaHarris',
  'twitter_screen_name': 'KamalaHarris',
  'twitter_user_id': '30354991',
  'announcement_date': 'January 21, 2019',
  'status': 'running'},
 {'name': 'Bernie Sanders',
  'party': 'Independent',
  'twitter_url': 'https://twitter.com/BernieSanders',
  'twitter_screen_name': 'BernieSanders',
  'twitter_user_id': '216776631',
  'announcement_date': 'February 19, 2019',
  'status': 'running'},
 {'name': 'Donald J. Trump',
  '

In [24]:
random_candidate = random.choice(filtered_candidates)
candidate_id = random_candidate["twitter_user_id"]

In [25]:
user_get = requests.get(f'https://api.twitter.com/1.1/statuses/user_timeline.json?id={candidate_id}&count=100', params = extended_payload, auth = auth)


In [26]:
user_json = user_get.json()

In [27]:
def filter_aged(list_element):
    date_string = list_element["created_at"]
    datetime_object = dt.datetime.strptime(date_string, "%a %b %d %H:%M:%S %z %Y")
    date_object = datetime_object.date()
    
    today_datetime = dt.datetime.utcnow()
    today_date = today_datetime.date()
    two_days_prior = today_date - dt.timedelta(days = 2)

    training_data = dt.date(2019, 8, 14)
    
    return ((date_object <= two_days_prior) and (date_object > training_data)) 

#     return date_object > training_data

In [28]:
user_filtered = list(filter(lambda x: filter_aged(x), user_json))

In [29]:
len(user_filtered)

81

In [30]:
tweet_selection = random.choice(user_filtered)

In [31]:
tweet_selection

{'created_at': 'Sun Aug 25 16:05:26 +0000 2019',
 'id': 1165656439492399104,
 'id_str': '1165656439492399104',
 'full_text': "John lived his life bound by a timeless code — decency, respect, and civility above all else. This week, as we remember his life and service, let's honor his legacy by living out that code. I hope you'll join me and engage in #ActsOfCivility. https://t.co/JwJcz87NVq",
 'truncated': False,
 'display_text_range': [0, 266],
 'entities': {'hashtags': [{'text': 'ActsOfCivility', 'indices': [226, 241]}],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/JwJcz87NVq',
    'expanded_url': 'https://joebiden.com/senator-john-mccain/',
    'display_url': 'joebiden.com/senator-john-m…',
    'indices': [243, 266]}]},
 'source': '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>',
 'in_reply_to_status_id': 1165656438615740423,
 'in_reply_to_status_id_str': '1165656438615740423',
 'in_reply_to_user_id': 939091,
 'in_reply

In [32]:
tweet_dict = {
    'full_text': tweet_selection["full_text"],
    'retweet_count': tweet_selection["retweet_count"],
    'favorite_count': tweet_selection['favorite_count'],
    'created_at': tweet_selection['created_at']
}

In [33]:
def convert_day(date_string):
    datetime_object = dt.datetime.strptime(date_string, "%a %b %d %H:%M:%S %z %Y")
    day = dt.datetime.strftime(datetime_object, "%A")
    return day
def convert_hour(date_string):
    datetime_object = dt.datetime.strptime(date_string, "%a %b %d %H:%M:%S %z %Y")
    hour = dt.datetime.strftime(datetime_object, "%H")
    return hour
def convert_month(date_string):
    datetime_object = dt.datetime.strptime(date_string, "%a %b %d %H:%M:%S %z %Y")
    month = dt.datetime.strftime(datetime_object, "%B")
    return month

In [34]:
tweet_dict['day'] = convert_day(tweet_dict['created_at'])
tweet_dict['hour'] = convert_hour(tweet_dict['created_at'])
tweet_dict['month']  = convert_month(tweet_dict['created_at'])

In [35]:
tweet_dict

{'full_text': "John lived his life bound by a timeless code — decency, respect, and civility above all else. This week, as we remember his life and service, let's honor his legacy by living out that code. I hope you'll join me and engage in #ActsOfCivility. https://t.co/JwJcz87NVq",
 'retweet_count': 748,
 'favorite_count': 5770,
 'created_at': 'Sun Aug 25 16:05:26 +0000 2019',
 'day': 'Sunday',
 'hour': '16',
 'month': 'August'}

In [36]:
tweet_list = [tweet_dict]

In [37]:
tweet_df = pd.DataFrame(tweet_list)
tweet_df.head()

,created_at,day,favorite_count,full_text,hour,month,retweet_count
0,Sun Aug 25 16:05:26 +0000 2019,Sunday,5770,John lived his life bound by a timeless code —...,16,August,748


In [38]:
import nltk
import re
import string
pd.set_option('display.max_colwidth', 100) # To extend column width

stopwords = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()

In [39]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [wn.lemmatize(word) for word in tokens if word not in stopwords]
    return text

In [40]:
from sklearn.feature_extraction.text import CountVectorizer

# NGramVectorizer
ngram_vect = CountVectorizer(ngram_range=(2,2), analyzer=clean_text)
ngram_vect.fit_transform(tweet_df['full_text'])

<1x23 sparse matrix of type '<class 'numpy.int64'>'
	with 23 stored elements in Compressed Sparse Row format>

In [41]:
ngram_vect.get_feature_names()

['actsofcivility',
 'bound',
 'civility',
 'code',
 'decency',
 'else',
 'engage',
 'honor',
 'hope',
 'httpstcojwjcz87nvq',
 'john',
 'join',
 'legacy',
 'let',
 'life',
 'lived',
 'living',
 'remember',
 'respect',
 'service',
 'timeless',
 'week',
 'youll']

In [42]:
import pickle
with open('rf_columns.pkl', 'rb') as f:
   columns_list = pickle.load(f)

In [43]:
null_list = []
for i in range(0, len(columns_list)):
    null_list.append(0)

In [44]:
X_features = dict(zip(columns_list, null_list))

In [45]:
X_features['retweet_count'] = tweet_dict['retweet_count']
X_features['favorite_count'] = tweet_dict['favorite_count']

select_month = tweet_dict['month']
select_day = tweet_dict['day']
select_hour = tweet_dict['hour']
X_features[f'month_{select_month}'] = 1
X_features[f'day_{select_day}'] = 1
X_features[f'hour_{select_hour}'] = 1

In [46]:
for word in ngram_vect.get_feature_names():
    if word in X_features.keys():
        X_features[word] += 1

In [47]:
len(list(X_features.values()))

20445

In [48]:
X_features = np.array(list(X_features.values())).reshape((1, 20445))

In [49]:
X_features.shape

(1, 20445)

In [50]:
X_features

array([[ 748, 5770,    0, ...,    0,    0,    0]])

In [52]:
X_sparse = csr_matrix(X_features)

In [53]:
X_sparse

<1x20445 sparse matrix of type '<class 'numpy.int32'>'
	with 25 stored elements in Compressed Sparse Row format>

In [54]:
X_sparse.toarray()

array([[ 748, 5770,    0, ...,    0,    0,    0]], dtype=int32)

In [56]:
scaler_filename = "rf_scaler.save"
scaler = joblib.load(scaler_filename) 

In [57]:
X_scaled = scaler.transform(X_sparse)

In [59]:
encoder = LabelEncoder()
encoder.classes_ = np.load('rf_classes.npy')

In [ ]:
prediction = model.predict_classes(X_scaled)
prediction_label = encoder.inverse_transform(prediction)
prediction_label